## 1. 문제 정의
어떤 특징을 갖는 사람이 생존할지를 예측하는 문제

## 2. 데이터 읽어오기

### * 캐글에서 받게되는 파일의 구성
- train.csv - 예측 모델을 만들기 위해 사용하는 학습셋
- test.csv - 예측 모델을 이용하여 예측할 탑승객 정보가 담긴 테스트셋
- sampleSubmission.csv - 제출시 사용할 수 있는 csv 파일

In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv', index_col = 0)
test = pd.read_csv('data/test.csv', index_col = 0)

## 3. 데이터 확인

In [2]:
train.tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


## 4. Feature engineering

### 숫자가 아닌 데이터를 숫자로 맵핑

In [3]:
train_test_data = [train, test]
def map_col(col, mapping_info):
    for dataset in train_test_data:
        dataset[col] = dataset[col].map(mapping_info)

In [4]:
map_col('Sex', {"male": 0, "female": 1})
map_col('Embarked', {"S": 0, "C": 1, "Q": 2})

In [5]:
train.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.338481,0.543351,-0.077221,-0.035322,0.081629,0.257307,0.108669
Pclass,-0.338481,1.000000,-0.131900,-0.369226,0.083081,0.018443,-0.549500,0.043835
Sex,0.543351,-0.131900,1.000000,-0.093254,0.114631,0.245489,0.182333,0.118593
Age,-0.077221,-0.369226,-0.093254,1.000000,-0.308247,-0.189119,0.096067,0.012186
SibSp,-0.035322,0.083081,0.114631,-0.308247,1.000000,0.414838,0.159651,-0.060606
Parch,0.081629,0.018443,0.245489,-0.189119,0.414838,1.000000,0.216225,-0.079320
Fare,0.257307,-0.549500,0.182333,0.096067,0.159651,0.216225,1.000000,0.063462
Embarked,0.108669,0.043835,0.118593,0.012186,-0.060606,-0.079320,0.063462,1.000000


In [6]:
train.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

### 불필요한 feature 제거

In [7]:
train_data = train.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
test_data = test.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
train_data.tail()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
887,0,2,0,27.0,0,0,13.00,0.0
888,1,1,1,19.0,0,0,30.00,0.0
889,0,3,1,NaN,1,2,23.45,0.0
890,1,1,0,26.0,0,0,30.00,1.0
891,0,3,0,32.0,0,0,7.75,2.0


### 결측값 처리하기
- train: 결측값 있는 경우 제외
- test: 결측값은 평균값으로 대체

In [8]:
train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [9]:
train_data.dropna(how = 'any', inplace = True)
train_data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [10]:
test_data.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [11]:
test_data['Age'].fillna(test_data['Age'].mean(), inplace = True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace = True)
test_data.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### 종속 변수와 독립 변수 설정

In [12]:
target = train_data.Survived
train_data.drop('Survived', axis = 1, inplace = True)

## 모델 적용

In [13]:
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import numpy as np

In [26]:
len_test = int(len(train_data)/10)
print(len_test)
def getScore(train_data, clf):        
    test_data   = train_data[-len_test:]; train_data = train_data[:-len_test]
    test_target = target[-len_test:];     train_target = target[:-len_test]
    clf.fit(train_data, train_target)    
    return (clf.predict(test_data) == np.array(test_target)).sum() / len_test

71


In [15]:
getScore(train_data, KNeighborsClassifier(n_neighbors = 13))

0.7605633802816901

In [29]:
scores = pd.DataFrame([
    getScore(train_data, KNeighborsClassifier(n_neighbors = 13)),
    getScore(train_data, DecisionTreeClassifier()),
    getScore(train_data, RandomForestClassifier(n_estimators=13)),
    getScore(train_data, GaussianNB()),
    getScore(train_data, SVC(gamma = 'auto'))],
    columns = ['score'],
    index = ['KNN', 'DT', 'RF', 'NB', 'SVM'])
scores

,score
KNN,0.760563
DT,0.816901
RF,0.901408
NB,0.816901
SVM,0.802817


In [35]:
clf = RandomForestClassifier(n_estimators=13)
clf.fit(train_data, target)
submission = pd.DataFrame({
        "PassengerId": test.index,
        "Survived": clf.predict(test_data)
    })
submission.tail()

,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0


In [34]:
submission.to_csv('submission.csv', index=False)